In [1]:
import pandas as pd

df = pd.read_csv('../data/copytrans.txt',
    encoding = "ISO-8859-1",
    chunksize=20000,
    engine='python',
    header=0,
    doublequote=True,
    index_col=False,
    names=[
        "TRANDATE,CTRANTP,BORBARCODE,LOC,TITLENO,COPYNO"
    ]
)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# jhoan = []
# 
# for chunk in df:      
#     # for now just add the chunk part
#     jhoan.append(chunk)
#     
# # concat the list into dataframe 
# df_jhoan = pd.concat(jhoan)
# df_jhoan.count()
# 
# data    14382068
# dtype: int64

In [2]:
chunk_list = []  # append each chunk df here 

def deleted_users(chuck):
    for index, row in chuck.iterrows():
        validation = row.str.contains("XXX_DELETEDUSER_", case=False)
        
        if validation.bool() == True:
            return 1
        else:
            return 0
        
    
def chunk_preprocessing(chuck):    
    replace_values = {
        'INSERT INTO copytrans \(trandate, ctrantp,borbarcode, titleno,copyno\) VALUES\( ' : '',
        '\);' : '',
        'XXX_DELETEDUSER_' : ''
    }
        
    chuck['deleted'] = deleted_users(chuck)  
    
    chuck.replace(
        to_replace={
            "TRANDATE,CTRANTP,BORBARCODE,LOC,TITLENO,COPYNO": replace_values
        },
        inplace=True,
        regex=True
    )
    
    return chuck

# Each chunk is in df format
for chunk in df:  
    # perform data filtering 
    chunk_filter = chunk_preprocessing(chunk)
    
    # for now just add the chunk part
    chunk_list.append(chunk_filter)
    
# concat the list into dataframe 
df_concat = pd.concat(chunk_list)

In [3]:
# df_concat.to_csv('copytrans_version1.csv', index=True, sep='*')
df_concat.count()

TRANDATE,CTRANTP,BORBARCODE,LOC,TITLENO,COPYNO    14382067
deleted                                           14382067
dtype: int64

In [4]:
df_copy = df_concat
df_copy.head()

,"TRANDATE,CTRANTP,BORBARCODE,LOC,TITLENO,COPYNO",deleted
0,"12/08/99,RET,9713183,BMC,20667,32376",1
1,"12/08/99,RET,9713183,BMC,139703,185804",1
2,"12/08/99,RET,9713183,BMC,141083,187596",1
3,"12/08/99,REN,9713183,BMC,23702,36458",1
4,"12/08/99,ISS,9713183,BMC,148308,196684",1


In [5]:
df_copy.columns.str.split(pat=",", expand=True)
df_copy.head()

,"TRANDATE,CTRANTP,BORBARCODE,LOC,TITLENO,COPYNO",deleted
0,"12/08/99,RET,9713183,BMC,20667,32376",1
1,"12/08/99,RET,9713183,BMC,139703,185804",1
2,"12/08/99,RET,9713183,BMC,141083,187596",1
3,"12/08/99,REN,9713183,BMC,23702,36458",1
4,"12/08/99,ISS,9713183,BMC,148308,196684",1


In [6]:
new_df = df_copy['TRANDATE,CTRANTP,BORBARCODE,LOC,TITLENO,COPYNO'].str.split(pat=",", expand=True)

In [7]:
new_df['DELETED'] = df_copy['deleted']

In [8]:
new_df = new_df.rename(
    columns={
        0: 'TRANDATE',
        1: 'CTRANTP',
        2: 'BORCODE',
        3: 'LOCATION',
        4: 'TITLENO',
        5: 'COPYNO',
    }
)

In [9]:
new_df.head()

,TRANDATE,CTRANTP,BORCODE,LOCATION,TITLENO,COPYNO,DELETED
0,12/08/99,RET,9713183,BMC,20667,32376,1
1,12/08/99,RET,9713183,BMC,139703,185804,1
2,12/08/99,RET,9713183,BMC,141083,187596,1
3,12/08/99,REN,9713183,BMC,23702,36458,1
4,12/08/99,ISS,9713183,BMC,148308,196684,1


In [10]:
new_df.to_csv('copytrans_modified.csv', index=False, sep='*')